[View in Colaboratory](https://colab.research.google.com/github/Manodhayan/dHaYaN/blob/master/Exam_Preparation_Online_Intern_Selection_Assignment.ipynb)

# Exam Preparation Online Intern Selection Assignment

### References


In [0]:
# https://gist.github.com/jmcarp/7105045
# https://github.com/shakkaist/Python/blob/master/Day2Session2/pdfconverter.py
# https://www.geeksforgeeks.org/working-with-pdf-files-in-python/

### Installing Required Packages

In [0]:
!pip install pdf2text
!pip install summa
!pip install gensim
!pip install pyldavis
!pip install PyPDF2

import nltk
nltk.download('stopwords')
nltk.download('wordnet')

### Accessing Data from GDrive

In [0]:
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
 
# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
file_list = drive.ListFile({'q': "'1mLNJza6w4ywUAJpHhKzucHqzJbQgoNL9' in parents and trashed=false"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))
  
pdf_data = drive.CreateFile({'id': '1Sg0Q1gtmXBrS9aYcKCm9R0GfvXCG0Nv_'})
pdf_data.GetContentFile("notes.pdf")

## PDF to Text Conversion

In [0]:
def pagewise_text_extraction(pdfFileObj):
  pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
  
  for page in range(pdfReader.numPages):
    pageObj = pdfReader.getPage(page)
    #print(page,pageObj.extractText())
    pagewise_text.append(pageObj.extractText())

In [0]:
import PyPDF2
import pandas as pd

pagewise_text=[]
pdfFileObj = open('notes.pdf', 'rb')
pagewise_text_extraction(pdfFileObj)

print(pdfReader.numPages)
 
# creating a page object
pageObj = pdfReader.getPage(2)
 
# extracting text from page
print(pageObj.extractText())
 
# closing the pdf file object
pdfFileObj.close()

## Preprocessing

### Cleaning Stage 1

In [0]:
pagewise_textdata = pd.DataFrame(data=pagewise_text,columns=["Text"])
pagewise_textdata["Page_No"]=""
pagenumber=1;
# Removing Page Header,Footer and Page numbers
template_words=['Java Basics','1996-2003 jGuru.com. All Rights Reserved.','-','1','2','3','4','5','6','7','8','9','10']
for index,row in pagewise_textdata.iterrows():
  pagewise_textdata.Page_No[index]=pagenumber
  pagenumber=pagenumber+1
  pagewise_textdata['Text']=pagewise_textdata['Text'].apply(lambda x: x.encode('ascii', 'ignore'))
  for words in template_words:
    pagewise_textdata['Text']=pagewise_textdata['Text'].replace(words,'',regex=True)

#Neglecting last pages as it doesn't contain any useful information
pagewise_textdata=pagewise_textdata[:-2]
#pagewise_textdata

### Cleaning Stage 2

In [363]:

pagewise_textdata["Meaningfull_Text"]=""

# Extracting meaningfull words
def clean_text(text):
        stopwords_set = set(stopwords.words("english"))
        filtered_words = [e.lower() for e in text.split() if len(e) >= 3]
        stopwords_less_words = [word for word in filtered_words if not word in stopwords_set]
        return ' '.join(stopwords_less_words)

for index,row in pagewise_textdata.iterrows():
  pagewise_textdata.Meaningfull_Text[index]=clean_text(pagewise_textdata.Text[index])

pagewise_textdata.head()

,Text,Page_No,Meaningfull_Text,Summary
0,Topics in this section include:! What makes ...,1,topics section include:! makes java programs p...,"makes java programs portable, secure, robust!\..."
1,are not portable.. The library. The Java cla...,2,portable.. library. java class library availab...,java class library available machine javarunti...
2,! Garbage collectionno bad addresses! Array ...,3,garbage collectionno bad addresses! array stri...,interfaces exceptionsjava program structurea f...
3,public class TrivialApplication { // args[0...,5,public class trivialapplication args[0] first ...,"class, effect,creates scoped symbol startingcl..."
4,threads so they do not burden system resourc...,6,threads burden system resources unnecessarily....,"therefore, forget call super.dispose() youover..."


## Text Summarization

In [364]:
from summa.summarizer import summarize
pagewise_textdata["Summary"]=""

# Create summaries for each page
for index,row in pagewise_textdata.iterrows():
  pagewise_textdata.Summary[index]=summarize(pagewise_textdata.Meaningfull_Text[index],ratio=0.3)
  if pagewise_textdata.Summary[index]=="":
    pagewise_textdata=pagewise_textdata.drop([index])

pagewise_textdata.reset_index(inplace=True,drop=True)  
pagewise_textdata.head()


,Text,Page_No,Meaningfull_Text,Summary
0,Topics in this section include:! What makes ...,1,topics section include:! makes java programs p...,"makes java programs portable, secure, robust!\..."
1,are not portable.. The library. The Java cla...,2,portable.. library. java class library availab...,java class library available machine javarunti...
2,! Garbage collectionno bad addresses! Array ...,3,garbage collectionno bad addresses! array stri...,interfaces exceptionsjava program structurea f...
3,public class TrivialApplication { // args[0...,5,public class trivialapplication args[0] first ...,"class, effect,creates scoped symbol startingcl..."
4,threads so they do not burden system resourc...,6,threads burden system resources unnecessarily....,"therefore, forget call super.dispose() youover..."


# Topic Modeling/ Keyword Extraction

###Cleaning Document

In [0]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
lemma = WordNetLemmatizer()
def clean(doc):
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

doc_clean = [clean(doc).split() for doc in pagewise_textdata.Summary]

###Prepare the Document to model

In [0]:
# Importing Gensim
import gensim
from gensim import corpora
# Creating the term dictionary of our courpus, where every unique term is assigned an index. 
dictionary = corpora.Dictionary(doc_clean)

# Converting list of documents (corpus) into Document Term Matrix using dictionary prepared above.
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
corpus=doc_term_matrix
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

###Model the Document

In [0]:
# Creating the object for LDA model using gensim library
Lda = gensim.models.ldamodel.LdaModel

# Running and Trainign LDA model on the document term matrix.
ldamodel = Lda(doc_term_matrix, num_topics=5, id2word = dictionary, passes=50)
ldamodel.save('model5.gensim')

###Results

In [369]:
print(ldamodel.print_topics(num_topics=5, num_words=3))

[(0, u'0.030*"parameter" + 0.023*"tag" + 0.016*"definition"'), (1, u'0.056*"java" + 0.031*"return" + 0.026*"program"'), (2, u'0.027*"new" + 0.021*"java" + 0.021*"garbage"'), (3, u'0.053*"applet" + 0.033*"method" + 0.021*"browser"'), (4, u'0.039*"comment" + 0.028*"string" + 0.023*"c"')]


###Visualisation

In [370]:
dictionary = gensim.corpora.Dictionary.load('dictionary.gensim')
corpus = pickle.load(open('corpus.pkl', 'rb'))
lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')

import pyLDAvis.gensim
lda_display = pyLDAvis.gensim.prepare(ldamodel, corpus, dictionary, sort_topics=True)
pyLDAvis.display(lda_display)